In [ ]:
import os
import lxml
import pytz
from lxml import etree
import datetime
from collections import defaultdict

tz = pytz.timezone('US/Pacific')
def get_xmls():
    directory = '.'
    files = [f for f in os.listdir(directory) if f.endswith('.xml')]
    data = [open(os.path.join(directory, f), 'rt').read() for f in files]
    return [etree.XML(d.encode('utf-8')) for d in data]

#r = get_xmls()[0].xpath("//*[name()='IntervalReading']")[0]

In [ ]:
#print(etree.tostring(r, pretty_print=True).decode('utf-8'))
def get_datapoint(node):
    start = int(node.xpath(".//*[name()='start']")[0].text)
    duration = int(node.xpath(".//*[name()='duration']")[0].text)
    value = int(node.xpath(".//*[name()='value']")[0].text)
    return (start, duration, value)

datapoints = [get_datapoint(node) for x in get_xmls() for node in x.xpath("//*[name()='IntervalReading']")]

In [ ]:
import json
jsonstr = open('./solar_production_20160813_to_20201108.json', 'rt').read()
solar = json.loads(jsonstr)['stats']
solardatadict = defaultdict(float)
for x in solar:
    tsdate = datetime.datetime.fromtimestamp(x['start_time'], tz).date()
    assert x['interval_length'] == 900
    for tstime, val in enumerate(x['production']):
        if tstime >= 96:
            assert val is None
        else:
            solardatadict[(tsdate, tstime)] = (0. if val is None else val)

In [ ]:
datadict = defaultdict(float)
dates = set()
for p in datapoints:
    start, duration, value = p
    assert duration == 900
    ts = datetime.datetime.fromtimestamp(start, tz)
    tsdate = ts.date()
    tstime = ts.hour * 4 + ts.minute // 15
    assert 0 <= tstime <= 95
    datadict[(tsdate, tstime)] = value
    dates.add(tsdate)

In [ ]:
mindate, maxdate = min(dates), max(dates)
mindate = datetime.date(2016,9,1)
ndates = (maxdate - mindate).days + 1
dates = [mindate + datetime.timedelta(days=i) for i in range(ndates)]

In [ ]:
import plotly.graph_objects as go
import datetime
import numpy as np

tstimes = [f'{i//4:02}:{15*(i%4):02}' for i in range(96)]
vals = np.zeros(shape=(96, ndates))
for k, v in datadict.items():
    tsdate, tstime = k
    dateind = (tsdate - mindate).days
    assert 0 <= dateind < ndates 
    vals[tstime, dateind] += v * 0
for k, v in solardatadict.items():
    tsdate, tstime = k
    dateind = (tsdate - mindate).days
    if 0 <= dateind < ndates:
        vals[tstime, dateind] += v * 4
    
fig = go.Figure(data=go.Heatmap(
        z=vals,
        x=dates,
        y=tstimes,
        #colorscale='Viridis'
    )
)

fig.update_layout(
    title='Power usage',
    xaxis_nticks=36,
    #autosize=False,
    #width=5000,
    #height=2000,
    #margin=dict(
    #    l=50,
    #    r=50,
    #    b=100,
    #    t=100,
    #    pad=4
    #),
)

fig.show()
